In [8]:
import os
import subprocess
import glob

# Prompt user for the backup directory path
backup_dir = input("Enter the full path to the backup directory: ").strip()
backup_file = os.path.join(backup_dir, "atfs.bak")

# Prompt for SQL Server connection info

sql_server ="100.103.17.32,1433"

'''sql_server = input("Enter SQL Server IP or hostname [default: 100.103.17.32,1433]: ").strip()
if not sql_server:
    sql_server = "100.103.17.32,1433"
elif "," not in sql_server:
    sql_server += ",1433"'''

# --- Hardcoded SQL credentials for testing ---
sql_user = "dbeaver"
sql_password = "dbeaver"
sql_script = r"D:\Projects\ATFS\Backup\restore_local.sql"

# Print connection details
print("\n--- SQL Server Restore Details ---")
print(f"Server: {sql_server}")
print(f"Authentication: SQL Login ({sql_user})")
print(f"SQL Script: {sql_script}")
print(f"Backup File: {backup_file}")
print("----------------------------------\n")

# Test connection
print("Testing connection to SQL Server...")
test_cmd = [
    "sqlcmd",
    "-S", sql_server,
    "-U", sql_user,
    "-P", sql_password,
    "-Q", "SELECT GETDATE();"
]

try:
    result = subprocess.run(test_cmd, shell=False, capture_output=True, text=True, timeout=10)
    if result.returncode != 0:
        print("❌ Failed to connect to SQL Server.")
        print("Error:", result.stderr.strip())
        input("Press Enter to exit...")
        raise SystemExit(1)
    else:
        print("✅ Connection successful.\n")
except Exception as e:
    print(f"❌ Connection test failed: {e}")
    input("Press Enter to exit...")
    raise SystemExit(1)

# Check if backup file exists
if not os.path.exists(backup_file):
    print(f"❌ Backup file not found: {backup_file}")
else:
    print("Restoring database...\n")

    restore_cmd = [
        "sqlcmd",
        "-S", sql_server,
        "-U", sql_user,
        "-P", sql_password,
        "-i", sql_script
    ]

    subprocess.run(restore_cmd, shell=False)

    # Delete used backup files
    for filename in ["atfs.bak", "atfs_log.bak"]:
        try:
            os.remove(os.path.join(backup_dir, filename))
        except FileNotFoundError:
            pass

    # Delete wildcard matches
    for file in glob.glob(os.path.join(backup_dir, "atfs_gdb*.bak")):
        os.remove(file)

# Pause before exit
input("Press Enter to exit...")



--- SQL Server Restore Details ---
Server: 100.103.17.32,1433
Authentication: SQL Login (dbeaver)
SQL Script: D:\Projects\ATFS\Backup\restore_local.sql
Backup File: /Users/brendanhall/Library/CloudStorage/OneDrive-Sewall/Projects/ATFS/_DB/DB_Restore/ATFS_Backups/atfs_bak_20250408_020000/atfs.bak
----------------------------------

Testing connection to SQL Server...
✅ Connection successful.

❌ Backup file not found: /Users/brendanhall/Library/CloudStorage/OneDrive-Sewall/Projects/ATFS/_DB/DB_Restore/ATFS_Backups/atfs_bak_20250408_020000/atfs.bak


''